## Data Ingestion Module

In [1]:
import os
%pwd

'c:\\Users\\Welcome\\Desktop\\TextSummarizer\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd


'c:\\Users\\Welcome\\Desktop\\TextSummarizer'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path


In [5]:
from src.text_summarizer.constants import *
from src.text_summarizer.utils.common import read_yaml, create_directories

## configuration updates

In [6]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from src.text_summarizer.logging import logger


## components

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded file and saved to {filename}")
        else:
            logger.info(f"File already exists ")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        # Check if the file is actually a zip file
        if not zipfile.is_zipfile(self.config.local_data_file):
            logger.error(f"File {self.config.local_data_file} is not a valid zip file")
            # Let's check what we actually downloaded
            with open(self.config.local_data_file, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read(500)  # Read first 500 characters
                logger.info(f"File content preview: {content}")
            raise ValueError(f"Downloaded file is not a valid zip file. It might be an HTML page or corrupted download.")
        
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to {unzip_path}")

    

In [10]:
# First, let's remove the corrupted file if it exists
import os
if os.path.exists("artifacts/data_ingestion/dataset.zip"):
    os.remove("artifacts/data_ingestion/dataset.zip")
    print("Removed corrupted file")

# Now try again with the corrected URL
config=ConfigurationManager()
data_ingestion_config=config.get_data_ingestion_config()
data_ingestion=DataIngestion(config=data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()

Removed corrupted file
[2025-07-03 12:01:10,595: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-03 12:01:10,597: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-03 12:01:10,598: INFO: common: created directory at: artifacts]
[2025-07-03 12:01:10,599: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-03 12:01:15,257: INFO: 2917796623: Downloaded file and saved to artifacts/data_ingestion/dataset.zip]
[2025-07-03 12:01:15,386: INFO: 2917796623: Extracted zip file to artifacts/data_ingestion]
